In [5]:
import pandas as pd
from src.recsys_baseline import VSKNN_STAN
from src.recsys_rl import rl_recommender
from src.evaluation import evaluate_sessions, MRR, HitRate, StrategicHybrid

## load datasets

In [26]:
train = pd.read_csv('data/processed datasets/diginetica/interactions_train_tr.txt',
                    sep='\t')
display(train.head())
display(train.shape)

,Type,SessionId,ItemId,Time,Date,Datestamp,TimeO,ItemSupport
0,view,1,9654,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:01:15.848000+00:00,74
1,view,1,33043,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:02:53.912000+00:00,41
2,view,1,32118,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:04:03.569000+00:00,19
3,view,1,12352,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:05:29.870000+00:00,79
4,view,1,35077,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:06:30.072000+00:00,47


(102272, 8)

In [27]:
test = pd.read_csv('data/processed datasets/diginetica/interactions_test.txt',
                    sep='\t')
display(test.head())
display(test.shape)

,Type,SessionId,ItemId,Time,Date,Datestamp,TimeO,ItemSupport
0,view,289,125013,1.464221e+09,2016-05-26,1.464221e+09,2016-05-26 00:00:18.301000+00:00,7
1,view,289,64068,1.464222e+09,2016-05-26,1.464221e+09,2016-05-26 00:14:07.735000+00:00,6
2,view,289,133346,1.464222e+09,2016-05-26,1.464221e+09,2016-05-26 00:14:38.934000+00:00,6
3,view,289,198930,1.464222e+09,2016-05-26,1.464221e+09,2016-05-26 00:18:48.607000+00:00,7
4,view,302,36202,1.464221e+09,2016-05-26,1.464221e+09,2016-05-26 00:00:45.583000+00:00,23


(64761, 8)

## Hybrid model

### 5 interactions

In [10]:
model_1 = rl_recommender(
    n_history=5,
    file_path=r'trained agents\diginetica\20240609_101255 - n_hist 5 state_size 5 action_size 568638.pth',
    state_size=5,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[5]) # after 5 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\diginetica\20240609_101255 - n_hist 5 state_size 5 action_size 568638.pth
Pretrained model successfully loaded


In [11]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  64761  actions in  14163  sessions
    eval process:  0  of  64761  actions:  0.0  %
    eval process:  1000  of  64761  actions:  1.5441392195920385  %
    eval process:  2000  of  64761  actions:  3.088278439184077  %
    eval process:  3000  of  64761  actions:  4.632417658776116  %
    eval process:  4000  of  64761  actions:  6.176556878368154  %
    eval process:  5000  of  64761  actions:  7.720696097960192  %
    eval process:  6000  of  64761  actions:  9.264835317552231  %
    eval process:  7000  of  64761  actions:  10.808974537144268  %
    eval process:  8000  of  64761  actions:  12.353113756736308  %
    eval process:  9000  of  64761  actions:  13.897252976328344  %
    eval process:  10000  of  64761  actions:  15.441392195920384  %
    eval process:  11000  of  64761  actions:  16.985531415512423  %
    eval process:  12000  of  64761  actions:  18.529670635104463  %
    eval process:  13000  of  64761  actions:  20.0738098546965  %
    eval proc

[('MRR@20: ',
  0.03677780020418993,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;\nPrecision@20: ;0.00015099531060755626;0.00015281159768993954;5.4900822055247393e-05;0.0003321443877621556;0.00011362130685135777;0.17405609516738915;0.17864503570325152;0.1690950624058209;0.1726554189162717;0.18951723363911796;0.1700765062331856;0.1851676362101092;0.15789756073310082;0.19458628334684996;0.15506227468712225;0.20675662248159662;0.16976829271214566;0.14500649828245035;0.11263987622683275;0.1606410788262824;0.15687830687830687;0.14377560752772978;0.16178013905286634;0.09774793108126441;0.14362745098039215;0.15619047619047616;0.13541666666666666;0.4089506172839506;0.048185941043083894;0.043333333333333335;0.06666666666666667;0.0365079365079365;0.5125;0.5277777777777778;0.3636363636363636;0.75;0.5;0.0;0.0;0.0;0.0;0.0;'),
 ('HitRate@20: ',
  0.10899640301988221,
  'Bin: ;\nHitRate@20: ;

### 10 interactions

In [28]:
model_1 = rl_recommender(
    n_history=10,
    file_path=r'trained agents\diginetica\20240609_122056 - n_hist 10 state_size 10 action_size 568638.pth',
    state_size=10,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[10]) # after 10 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\diginetica\20240609_122056 - n_hist 10 state_size 10 action_size 568638.pth
Pretrained model successfully loaded


In [29]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  64761  actions in  14163  sessions
    eval process:  0  of  64761  actions:  0.0  %
    eval process:  1000  of  64761  actions:  1.5441392195920385  %
    eval process:  2000  of  64761  actions:  3.088278439184077  %
    eval process:  3000  of  64761  actions:  4.632417658776116  %
    eval process:  4000  of  64761  actions:  6.176556878368154  %
    eval process:  5000  of  64761  actions:  7.720696097960192  %
    eval process:  6000  of  64761  actions:  9.264835317552231  %
    eval process:  7000  of  64761  actions:  10.808974537144268  %
    eval process:  8000  of  64761  actions:  12.353113756736308  %
    eval process:  9000  of  64761  actions:  13.897252976328344  %
    eval process:  10000  of  64761  actions:  15.441392195920384  %
    eval process:  11000  of  64761  actions:  16.985531415512423  %
    eval process:  12000  of  64761  actions:  18.529670635104463  %
    eval process:  13000  of  64761  actions:  20.0738098546965  %
    eval proc

[('MRR@20: ',
  0.008672161262000221,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;\nPrecision@20: ;6.058814838734347e-05;0.00015004354145237291;0.000318469556641493;0.00020998286364716626;0.00011060050745959066;9.7911227154047e-05;0.00012957542814334743;0.00033783783783783786;0.0;7.603406326034063e-05;0.1700765062331856;0.1851676362101092;0.15789756073310082;0.19458628334684996;0.15506227468712225;0.20675662248159662;0.16976829271214566;0.14500649828245035;0.11263987622683275;0.1606410788262824;0.15687830687830687;0.14377560752772978;0.16178013905286634;0.09774793108126441;0.14362745098039215;0.15619047619047616;0.13541666666666666;0.4089506172839506;0.048185941043083894;0.043333333333333335;0.06666666666666667;0.0365079365079365;0.5125;0.5277777777777778;0.3636363636363636;0.75;0.5;0.0;0.0;0.0;0.0;0.0;'),
 ('HitRate@20: ',
  0.026226333056642556,
  'Bin: ;\nHitRate@20: ;',
  '

### 15 interactions

In [30]:
model_1 = rl_recommender(
    n_history=15,
    file_path=r'trained agents\diginetica\20240609_123637 - n_hist 15 state_size 15 action_size 568638.pth',
    state_size=15,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[15]) # after 15 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\diginetica\20240609_123637 - n_hist 15 state_size 15 action_size 568638.pth
Pretrained model successfully loaded


In [31]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  64761  actions in  14163  sessions
    eval process:  0  of  64761  actions:  0.0  %
    eval process:  1000  of  64761  actions:  1.5441392195920385  %
    eval process:  2000  of  64761  actions:  3.088278439184077  %
    eval process:  3000  of  64761  actions:  4.632417658776116  %
    eval process:  4000  of  64761  actions:  6.176556878368154  %
    eval process:  5000  of  64761  actions:  7.720696097960192  %
    eval process:  6000  of  64761  actions:  9.264835317552231  %
    eval process:  7000  of  64761  actions:  10.808974537144268  %
    eval process:  8000  of  64761  actions:  12.353113756736308  %
    eval process:  9000  of  64761  actions:  13.897252976328344  %
    eval process:  10000  of  64761  actions:  15.441392195920384  %
    eval process:  11000  of  64761  actions:  16.985531415512423  %
    eval process:  12000  of  64761  actions:  18.529670635104463  %
    eval process:  13000  of  64761  actions:  20.0738098546965  %
    eval proc

[('MRR@20: ',
  0.002357908330076402,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;\nPrecision@20: ;6.894819267309485e-05;0.00012662508626301805;7.676066491129142e-05;0.0001047671878672252;0.0004126522095350119;0.0002279414812051888;0.0003884953121873626;0.0;0.0;0.00040551500405515;0.0;0.002976190476190476;0.0;0.0;0.0053475935828877;0.20675662248159662;0.16976829271214566;0.14500649828245035;0.11263987622683275;0.1606410788262824;0.15687830687830687;0.14377560752772978;0.16178013905286634;0.09774793108126441;0.14362745098039215;0.15619047619047616;0.13541666666666666;0.4089506172839506;0.048185941043083894;0.043333333333333335;0.06666666666666667;0.0365079365079365;0.5125;0.5277777777777778;0.3636363636363636;0.75;0.5;0.0;0.0;0.0;0.0;0.0;'),
 ('HitRate@20: ',
  0.007352069251749081,
  'Bin: ;\nHitRate@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23

### 100 interactions
In other words, the model almost only uses the first model to predict next interactions with items.

In [32]:
model_1 = rl_recommender(
    n_history=100,
    file_path=r'trained agents\diginetica\20240609_134910 - n_hist 100 state_size 100 action_size 568638.pth',
    state_size=100,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[100]) # after 100 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\diginetica\20240609_134910 - n_hist 100 state_size 100 action_size 568638.pth
Pretrained model successfully loaded


In [33]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  64761  actions in  14163  sessions
    eval process:  0  of  64761  actions:  0.0  %
    eval process:  1000  of  64761  actions:  1.5441392195920385  %
    eval process:  2000  of  64761  actions:  3.088278439184077  %
    eval process:  3000  of  64761  actions:  4.632417658776116  %
    eval process:  4000  of  64761  actions:  6.176556878368154  %
    eval process:  5000  of  64761  actions:  7.720696097960192  %
    eval process:  6000  of  64761  actions:  9.264835317552231  %
    eval process:  7000  of  64761  actions:  10.808974537144268  %
    eval process:  8000  of  64761  actions:  12.353113756736308  %
    eval process:  9000  of  64761  actions:  13.897252976328344  %
    eval process:  10000  of  64761  actions:  15.441392195920384  %
    eval process:  11000  of  64761  actions:  16.985531415512423  %
    eval process:  12000  of  64761  actions:  18.529670635104463  %
    eval process:  13000  of  64761  actions:  20.0738098546965  %
    eval proc

[('MRR@20: ',
  0.00028014935605370707,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;\nPrecision@20: ;0.0005304994234308809;0.00010172160460181947;0.00012364168113845824;9.228596232453393e-05;0.0005733394165931867;7.459903021260725e-05;0.0;0.0;4.7673531655225015e-05;0.00030992932452786467;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.002008032128514056;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.16666666666666666;0.0;0.5;0.0;0.0;0.0;0.0;0.0;0.0;'),
 ('HitRate@20: ',
  0.0013241630103956678,
  'Bin: ;\nHitRate@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;\nHitRate@20: ;0.0019063757678457953;0.0008010413537598879;0.0012785907089075152;0.0009850275807722615;0.002150537634408602;0.0011189854531891085;0.0;0.0;0.0009057971014492754;0.0036496350364963502;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0120481927710